# Two-Handed Optimization

A custom meta-algorithm I like to call "Two-Handed optimization". We have two sub-algorithms (in this case random search and gradient descent) and we vary which we use depending on how well each algorithm is performing: the better it's doing, the more we use it. The theory is that by having one exploratory algorithm (RS) and one exploitative algorithm (GD) we can better solve the explore-exploit problem by automating the decision as to which algorithm to use with a mesa optimizer.

In [1]:
import random as r
import numpy as np

## Objective Functions

In this notebook we'll test the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

If you want a different objective function, check out Objectives.ipynb.

In [2]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

## Two-Handed

In [18]:
class TwoHanded:
    def __init__(
        self,
        ranges,
        leftPoints = 1,
        rightPoints = 1
    ):
        self.ranges = ranges
        self.leftPoints = leftPoints
        self.rightPoints = rightPoints
        
    def selectLeftOrRight(self):
        probLeft = self.leftPoints / (self.leftPoints + self.rightPoints)
        if r.uniform(0, 1) < probLeft:
            return("L")
        else:
            return("R")

    #Random search's hyothesis selector.
    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)

    #One iteration of gradient descent
    def gradientStep(self, objectiveFunction, currentX, dx = 10 ** (-10), learnRate = 0.01):
        gradientVector = []
        for i in range(len(currentX)):
            cloneX = [x for x in currentX]
            Y1 = objectiveFunction(cloneX)
            cloneX[i] += dx
            Y2 = objectiveFunction(cloneX)
            gradientVector.append((Y2 - Y1)/dx)
        outputX = [x for x in currentX]
        for i in range(len(outputX)):
            outputX[i] += gradientVector[i] * learnRate
        return(outputX)
    
    def optimize(self, objectiveFunction, numIterations = 25000):
        currentX = None
        currentY = -float("inf")
        for i in range(numIterations):
            leftRightDecision = self.selectLeftOrRight()
            if leftRightDecision == "L":
                newX = self.generateRandomHypothesis()
                newY = objectiveFunction(newX)
                if newY > currentY:
                    currentY = newY
                    currentX = newX
                    self.leftPoints += 1
                else:
                    self.rightPoints += 1
            else:
                if currentX == None:
                     currentX = [np.mean(x) for x in self.ranges]
                newX = self.gradientStep(objectiveFunction, currentX)
                newY = objectiveFunction(newX)
                if newY > currentY:
                    currentY = newY
                    currentX = newX
                    self.rightPoints += 1
                else:
                    self.leftPoints += 1
        return(currentX)
th = TwoHanded([[1, 120], [1, 180]])
print(th.optimize(SEMF))

[26.397531429833865, 32.693510329091445]
